# <center>پیدا کردن ربات</center>

<div dir="rtl" style="font-size:18px">
احتمالا کارتون WALL-E را بخاطر دارید. رباتی که توانست زمین را نجات دهد و باقیمانده نسل انسان‌ها را به زمین برگرداند. پس از بازگشت انسان‌ها به زمین و پس از محبوبیت زیاد WALL-E، او دوستان زیادی پیدا کرد. بهترین دوست او WALL-F یکی از بازیگوش‌ترین ربات‌هایی است که تاحالا دیده‌اید.
<br>
یکی از دفعاتی که WALL-F به بیرون رفته بود، راهی به درون یک هزار‌تو میابد. تصویر زیر از او در این هزارتو گرفته شده است.
</div>
<br>
<br>
<center>

![WALL-F Ai-Generated](Assets/robot.jpg)

</center>

<div dir="rtl">
در ادامه میخواهیم با روش‌های مختلفی پیدا کنیم که در حال حاضر ربات در چه خانه‌ای قرار دارد. در هرکدام از تست‌کیس‌هایی که لازم است بررسی کنید. خانه 0 و 0 خانه بالا و چپ هزارتو خواهد بود. در ادامه برای هر خانه مشخص خواهد شد که دیواری به هرکدام از جهات دارد یا خیر. 
به طور مثال عبارت
''R L''
به این معنی است که این خانه فقط به چپ و راست خود راه دارد. همچنین شماره‌گذاری خانه‌ها به صورت زیر است.
در هر مرحله او به شما میگوید که در کدام خانه از جدول است. عددی که ربات به شما میگوید یک عدد تصادفی است که در یک ماتریس به شما داده میشود. عدد سطر 
$i$ 
و ستون 
$j$ 
احتمال این را نشان می‌دهد که ربات در خانه 
$i$ 
باشد اما خانه 
$j$ 
را شما گزارش دهد. این ربات در پایان هر مرحله با احتمال برابر به یکی از خانه‌های مجاور میرود. همینطور خانه شروع این ربات هم توزیعی یکنواخت دارد.
</div>
<br>
<br>
<center>

![grid](Assets/grid.jpg)

</center>

<div dir="rtl">
در ابتدا برای پیاده‌سازی ۲ الگوریتمی که در ادامه خواهید دید به آبجکتی از نوع Grid نیاز خواهید داشت. عملکرد این آبجکت را میتوانید از مسیر Assets/helpers.py ببینید. به طور مثال هزارتو زیر را در نظر بگیرید.
</div>

<center>

![maze](Assets/grid2.jpg)

</center>

<div dir="rtl">
grid متناظر با این هزارتو را میتوانید از طریق تابع get_sample_grid بگیرید و با توابع آن آشنا شوید. در صورت لزوم میتوانید هر تابع دلخواهی به این کلاس اضافه کنید. برای اطلاع از تعداد سطرهای grid میتوانید از grid.n و برای اطلاع از تعداد ستون‌ها میتوانید از grid.m استفاده کنید.
</div>

In [44]:
from Assets.helpers import get_sample_grid
import numpy as np
# 'name' = 'محمدشایان شعبانی'
# 'student_number' = '400105069'

In [8]:
grid = get_sample_grid()

# Particle Filtering

<div dir="rtl">
در این قسمت باید آخرین خانه‌ای که این ربات در آن قرار دارد را با استفاده از الگوریتم Particle Filtering پیدا کنید. برای اینکار در ابتدا تعداد Particle هایی با تعدادی از اردر تعداد خانه‌های هزارتو بسازید. در پایان هر مرحله جمعیت را با توجه به وزن آنها نصف کنید و ذراتی جدید متناسب با تعداد ذرات درون هرخانه بسازید. در پایان خانه‌ای که بیشترین ذرات در آن قرار دارد را برگردانید.
</div>

In [18]:
def PF(grid, prob, obs):
    """
    input:

    grid: object of type Grid
    obs: array of ints, each one is an observation

    output: 
    number of the last cell
    """
    particles = np.random.randint(0, grid.m * grid.n, size = 11 * grid.m * grid.n)
    for ob in obs:
        for i in range(len(particles)):
            choice = np.random.choice(grid.moves[particles[i] // grid.m][particles[i] % grid.m])
            if choice == 'U':
                particles[i] -= grid.m
            elif choice == 'R':
                particles[i] += 1
            elif choice == 'D':
                particles[i] += grid.m
            elif choice == 'L':
                particles[i] -= 1
        weights = np.array([prob[particle, ob] for particle in particles])
        weights /= sum(weights)
        particles = np.random.choice(particles, size = 11 * grid.m * grid.n, p = weights)
    particles_number = np.zeros(11 * grid.m * grid.n)
    for particle in particles:
        particles_number[particle] += 1
    return particles_number.argmax()

In [19]:
from Assets.helpers import PF_checker
PF_checker(PF)

PF PASSED!


# HMM

<div dir="rtl">
در این قسمت غیر از اینکه باید بفهمیم WALL-F در کدام خانه قرار دارد. میخواهیم بدانیم که محتمل‌ترین مسیری که طی کرده‌ است چه مسیری است. برای اینکار الگوریتم viterbi را پیاده‌سازی کنید.
</div>

In [42]:
def viterbi(grid, prob, obs):
    num_cells = grid.m * grid.n
    transition_mat = np.zeros((num_cells, num_cells))
    for i in range(num_cells):
        for j in grid.get_neighbors(i):
            transition_mat[i, j] = 1
        transition_mat[i] = transition_mat[i] / np.sum(transition_mat[i])
    """
    input:

    grid: object of type Grid
    obs: array of ints, each one is an observation

    output: 
    array of cell numbers, same shape as obs
    """
    
    num_observations = len(obs)
    num_states = grid.m * grid.n

    
    state_probabilities = [[float(0) for i in range(num_states)] for i in range(num_observations)]

    
    arg_max_state = [[0 for i in range(num_states)] for i in range(num_observations)]

    for i in range(len(obs)):
        for state in range(grid.m * grid.n):
            if i == 0:
                state_probabilities[i][state] = prob[state][obs[i]]
                state_probabilities[i][state] /= grid.m
                state_probabilities[i][state] /= grid.n
            else:
                maximum = -1000000000
                maximum_index = -1
                for j in range(grid.m * grid.n):
                    if state_probabilities[i - 1][j] * transition_mat[j, state] > maximum:
                        maximum = state_probabilities[i - 1][j] * transition_mat[j, state]
                        maximum_index = j
                arg_max_state[i][state] = maximum_index
                state_probabilities[i][state] = maximum * prob[state][obs[i]]
    maximum = -1000000000
    maximum_index = -1
    for j in range(grid.m * grid.n):
        if state_probabilities[-1][j] > maximum:
            maximum = state_probabilities[-1][j]
            maximum_index = j
    dp = [maximum_index]
    for i in range(len(obs) - 1, 0, -1):
        dp.append(arg_max_state[i][dp[-1]])
    dp.reverse()
    return dp

In [43]:
from Assets.helpers import viterbi_checker
viterbi_checker(viterbi)

VITERBI PASSED!
